<a href="https://colab.research.google.com/github/Meghraj-Webllisto/code/blob/main/nlp_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-8aa701fd-78e1-f41b-1b6d-64e9a1b1a228)


In [2]:
# Let Download some helper function
! wget http://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-05-24 10:58:08--  http://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py [following]
--2021-05-24 10:58:08--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-05-24 10:58:08 (112 MB/s) - ‘helper_f

In [3]:
# Import Series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [4]:
# Download the dataset
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# unzip data
unzip_data("nlp_getting_started.zip")

--2021-05-24 10:58:10--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.62.128, 172.217.164.176, 172.253.115.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.62.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2021-05-24 10:58:10 (109 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
# reading csv using pandas
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42) # shuflle with random state=42 for reproducebility
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
test_df.head()
# test doesn't have target column

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
# how many examples of each class
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

1 = real disaster tweet
0 = not a real disaster tweet

In [9]:
# How many smapels total?
print(f"Total training smapels: {len(train_df)}" )
print(f"total test smaples: {len(test_df)}")
print(f"Total Samples: {len(train_df) + len(test_df)}")

Total training smapels: 7613
total test smaples: 3263
Total Samples: 10876


In [10]:
# Let visualizer some training example
import random 
random_index = random.randint(0, len(train_df)-5)  # Creaete radom index not highter than total number of samples
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
    _, text, target = row
    print(f"Target: {target}", "(real disaster)" if target >0 else "(not real disaster)")
    print(f"Text: \n{text}\n")
    print(f"---\n")

Target: 1 (real disaster)
Text: 
Malaysian Officials Say Debris Found on Reunion Island Is From #MH370. @BillNeelyNBC reports: http://t.co/r6kZSQDghZ

---

Target: 1 (real disaster)
Text: 
@rachelcaine The weatherit needs to make it minds up. First snow tornadoes now would you say a heat wave?

---

Target: 0 (not real disaster)
Text: 
Upheaval high note for bush opera http://t.co/aWPU0gaE0b #Sydney #News #Aus

---

Target: 1 (real disaster)
Text: 
RT @NLM_DIMRC: A deluge of resources on #floods for medical providers cleanup workers &amp; more at: http://t.co/aUoeyIRqE6

---

Target: 0 (not real disaster)
Text: 
Thinking about getting a demo car with a friend and joining the demolition derby in kenosha

---



# split data into trian and validation sets

In [11]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # 10% percent only for validations
                                                                            random_state=42) # for reproducebility

In [12]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [13]:
# view some sampels 
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

# Converting text into number because it need numbers to run

In [14]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization
text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None,
                                    output_mode="int",
                                    output_sequence_length=None,
                                    pad_to_max_tokens=True)


In [15]:
# find the average number of tokens(words) in training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [16]:
# setup text vectorization variables
max_vocab_length = 10000
max_length =15

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [17]:
text_vectorizer.adapt(train_sentences)

In [18]:
# Creating a sample sentence 
smaple_sentence = " There's  a flood in my area."
text_vectorizer([smaple_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 408,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [19]:
# unqiue workds in vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tolens ([unk] for unknown words)
bottom_5_words = words_in_vocab[-5:]
print(f"Numbner of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 Most common words: {top_5_words}")
print(f"Bottom 5 Most common words: {bottom_5_words}")

Numbner of words in vocab: 10000
Top 5 Most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 Most common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


# Creating an Embeding using Embedding layer

In [20]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             embeddings_initializer='uniform',
                             input_length=max_length)
embedding

In [21]:
# lets check one sentence in embedding
random_sentences =  random.choice(train_sentences)

print(f"original sentence: {random_sentences}")
sample_embedding_output = embedding(text_vectorizer([random_sentences]))
sample_embedding_output

original sentence: I liked a @YouTube video from @sqwizzix http://t.co/GGqCz9AB6u Call of Duty: ÛÏThe Piano EntertainerÛ Ep. 9 ÛÒ Musicians Collide!


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.03174826,  0.01158581,  0.00996625, ..., -0.02533834,
         -0.03915069,  0.04471805],
        [-0.03315306,  0.02884961, -0.04337993, ..., -0.00401814,
          0.03109732,  0.03304734],
        [ 0.00385405,  0.00966021,  0.02291754, ...,  0.02427837,
          0.0163605 ,  0.0423292 ],
        ...,
        [ 0.02526024, -0.03154299,  0.04741411, ..., -0.01185925,
          0.04993299, -0.00743997],
        [-0.03934727,  0.02316416, -0.03660525, ..., -0.01022897,
         -0.02393894, -0.04484742],
        [ 0.0151295 ,  0.03419603,  0.02264159, ..., -0.03119685,
          0.04130092,  0.04536611]]], dtype=float32)>

In [22]:
type(sample_embedding_output)

tensorflow.python.framework.ops.EagerTensor

In [23]:
sample_embedding_output[0][-2]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([-0.03934727,  0.02316416, -0.03660525, -0.01670171,  0.00292802,
        0.03768024,  0.0345413 ,  0.0417153 , -0.00675485, -0.04380372,
       -0.01049144,  0.02630904, -0.01389908, -0.02584611, -0.04274907,
       -0.04907848, -0.01506655,  0.01589736, -0.01081372,  0.01571088,
        0.03448464, -0.0324118 , -0.00297669,  0.02135867, -0.03960029,
       -0.01683478, -0.0081922 , -0.01378689, -0.04670122,  0.03760168,
        0.02323138, -0.03776643,  0.01622428,  0.03877488,  0.04728338,
       -0.03817146,  0.03731562, -0.04270599, -0.01726912,  0.01677345,
       -0.04429947,  0.04612657, -0.00360105,  0.04525885, -0.04241545,
       -0.02366905, -0.0409417 ,  0.02648741,  0.03140214,  0.02927352,
        0.02682981,  0.02163167,  0.00640206, -0.00723679,  0.04304523,
        0.03044847, -0.00305087, -0.01877788, -0.00396074,  0.03108729,
       -0.04338261, -0.02178837, -0.04950574,  0.0234058 ,  0.00891101,
       -0.044796

Embedding is ready , let build model if we can , there is no we just me MRJ

# THE NAIVE BASE model(baseline)
![](https://cdn.journaldev.com/wp-content/uploads/2020/10/NAIVE-BAYES-1.png)

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Creating tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()),
                    ("clf", MultinomialNB()),
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

The benifit of shallow model like MultinomialNB is that trianing is very fast.

In [25]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"baseline model has an accuracy of: {baseline_score*100:.2f}%")

baseline model has an accuracy of: 79.27%


In [26]:
# makeing prediction
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

# Creating an evaluation function for our model experiments
it will evaluate
accuracy 

precision

Recall

F1-score


In [30]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
    """
    Calculates model accuracy, precision, recall and f1 score of a binary classification model.
    Args :
    y_true : True labels in the form of a 1 d array
    y_pred : predicted labels in the form of a 1d array
    Return a dictionary of accuracy, precision, recall, f1-score.
    """
    # calcualte model accuracy
    model_accuracy = accuracy_score(y_true, y_pred) * 100
    # calculate model precision, recall and f1 score using "weighted" average
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred,  average="weighted")
    model_results = {"accuracy" : model_accuracy,
                     "precision" : model_precision,
                     "recall": model_recall,
                     "f1": model_f1}
    return model_results

In [31]:
# get baseline results
baseline_results = calculate_results(y_true=val_labels, y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

# model 1:A simple dense model

In [32]:
from helper_functions import create_tensorboard_callback
SAVE_DIR = "model_logs" # tensorboard log dirs

In [35]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # input are 1 dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # Create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding 
outputs = layers.Dense(1, activation='sigmoid')(x)
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # consturct the model.

In [41]:
# compile the model 
model_1.compile(loss="binary_crossentropy",
               optimizer=tf.keras.optimizers.Adam(),
               metrics=['acc'])

In [42]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d_1 ( (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [43]:
# fit a model
model_1_history = model_1.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20210524-112435
Epoch 1/5
215/215 [==============================] - 7s 16ms/step - loss: 0.6526 - acc: 0.6390 - val_loss: 0.5354 - val_acc: 0.7559
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.4611 - acc: 0.8151 - val_loss: 0.4711 - val_acc: 0.7848
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3537 - acc: 0.8607 - val_loss: 0.4588 - val_acc: 0.7900
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.2806 - acc: 0.8942 - val_loss: 0.4653 - val_acc: 0.7887
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.2314 - acc: 0.9207 - val_loss: 0.4856 - val_acc: 0.7913
